In [1]:
import fairseq

In [2]:
from fairseq.models.roberta import RobertaModel

roberta = RobertaModel.from_pretrained('/projects/deepgreen/pstjohn/roberta_base_checkpoint/',
                                       checkpoint_file='checkpoint_best.pt')

state = fairseq.checkpoint_utils.load_checkpoint_to_cpu('/projects/deepgreen/pstjohn/roberta_base_checkpoint/checkpoint_best.pt')

In [3]:
state.keys()

dict_keys(['cfg', 'args', 'model', 'optimizer_history', 'extra_state', 'last_optimizer_state'])

In [4]:
import numpy as np
import scipy.sparse
from scipy.special import logit

train_annotations = scipy.sparse.load_npz('/projects/deepgreen/pstjohn/swissprot_go_annotation/fairseq_swissprot/label/train.npz')
epsilon = 1E-8
initial_bias = logit((np.asarray(train_annotations.sum(0)).squeeze() / train_annotations.shape[0]) + epsilon)
initial_bias

array([ -9.11901444,  -8.10568583,  -7.92012871, ..., -13.15707456,
       -13.15707456, -13.15707456])

In [5]:
roberta.model.register_classification_head('go_prediction', num_classes=initial_bias.shape[0])

In [6]:
bias = roberta.model.classification_heads['go_prediction'].out_proj.bias.data
roberta.model.classification_heads['go_prediction'].out_proj.bias.data = bias.new(initial_bias)

In [7]:
new_state = dict(state)
new_state['model'] = roberta.model.state_dict()

In [8]:
import torch
torch.save(new_state, '/projects/deepgreen/pstjohn/roberta_base_checkpoint/roberta.base_with_go_bias.pt')

In [9]:
roberta = RobertaModel.from_pretrained('/projects/deepgreen/pstjohn/roberta_base_checkpoint/',
                                       checkpoint_file='roberta.base_with_go_bias.pt')

In [10]:
roberta.model.classification_heads.keys()

odict_keys(['go_prediction'])